In [26]:
import requests
import json
from datetime import date, timedelta, datetime
from dateutil import parser
import sys

In [27]:
study_id = 'StudyHack'
ae_domain_id ='ae'
cm_domain_id = 'cm'
base_url = 'https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/'

In [28]:
ae_subject_id_url = base_url + study_id+'/'+ae_domain_id+'/subject/list'
cm_subject_id_url = base_url + study_id+'/'+cm_domain_id+'/subject/list'

In [29]:
print(ae_subject_id_url)

https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/list


In [30]:
print(cm_subject_id_url)

https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/cm/subject/list


In [31]:
try:
    ae_response_data1 = requests.get(ae_subject_id_url)
    ae_response_value1 = ae_response_data1.json()
    ae_subject_ids = ae_response_value1["data"]
except:
    sys.exit("Unable to get the values!")

In [32]:
try:
    cm_response_data1 = requests.get(cm_subject_id_url)
    cm_response_value1 = cm_response_data1.json()
    cm_subject_ids = cm_response_value1["data"]
except:
    sys.exit("Unable to get the values!")

In [33]:
# Patient and row for which Medication is given prior to Adverse Event
# Function will get data at subject_id level 
# Identify discrepancy and return discrepancy_list as a list 

def type1_check(ae_subject_data,cm_subject_data):
    discrepancy_list = []

    #Iterate through cm_subject_data
    for cm_row in cm_subject_data:
        cm_data = cm_row.json()
        cmaeno = cm_data["cmaeno"]
        cmstdat = cm_data["cmstdat"]
        cmendat = cm_data["cmendat"]
        cmtrt = cm_data["cmtrt"]
        
        #Iterate through ae_subject_data
        for ae_row in ae_subject_data:
            ae_data = ae_row.json()
            aespid = ae_data["aespid"]
            aestdat = ae_data["aestdat"]
            aeendat = ae_data["aeendat"]
            aeterm = ae_data["aeterm"]
            
            if aespid == cmaeno:
                if cmstdat < aestdat:
                    payload={}
                    payload['formname'] = cm_data["formname"]
                    payload['formid'] = cm_data["formid"]
                    payload['formidx']= cm_data["formidx"]
                    payload['type'] = 'TYPE1'
                    payload['subjectid'] = cm_data["subjid"]
                    discrepancy_list.append(payload)
            
    return discrepancy_list

In [34]:
for subject in ae_subject_ids:
    ae_subject_data_url = base_url + "/" + study_id+"/" + ae_domain_id + "/subject/" + str(subject)+ "/list"
    ae_response_data2 = requests.get(ae_subject_data_url)
    ae_response_value2 = ae_response_data2.json() 
    ae_subject_data = ae_response_value2["data"]
    cm_subject_data_url = base_url + "/" + study_id+"/" + ae_domain_id + "/subject/" + str(subject)+ "/list"
    cm_response_data2 = requests.get(cm_subject_data_url)
    cm_response_value2 = cm_response_data2.json() 
    cm_subject_data = cm_response_value2["data"]
    
    discrepancy_list = type1_check(ae_subject_data,cm_subject_data)
    print(discrepancy_list)

AttributeError: 'dict' object has no attribute 'json'